In [ ]:
import pandas as pd
import numpy as np
from numpy import load
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Concatenate

from keras.regularizers import l1_l2
from tensorflow import keras
import shutil
from collections import defaultdict
import json
from pathlib import Path
import os
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from tensorflow.keras import backend as K

In [ ]:
path_images="./images.npz"
path_cities="./cities.npz"
path_class="./classes.npz"

In [ ]:
from numpy import load
dict_image_data = load(path_images)
image_data = dict_image_data['arr_0']
dict_class_data = load(path_class)
class_data = dict_class_data['arr_0']
dict_city_data = load(path_cities)
city_data = dict_city_data['arr_0']

In [ ]:
# all 512 images resized to 128
data_512 = image_data[:,0]
# all 512 images cropped to 256 images and resized to 128
data_256 = image_data[:,1]
# all 512 images cropped to 128 images
data_128 = image_data[:,2]

In [ ]:
city_data.shape, data_512.shape, image_data.shape

((28195,), (28195, 128, 128, 3), (28195, 3, 128, 128, 3))

In [ ]:
len(np.unique(class_data))

18

### Tranformation of labels to onehot decoding

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()

train__y_labels_city = label_as_binary.fit_transform(city_data)
train__y_labels_class = label_as_binary.fit_transform(class_data)

### Split the data set to train and test


In [ ]:
#City
train_X_512_ci,valid_X_512_ci,train_label_512_ci,valid_label_512_ci = train_test_split(data_512, train__y_labels_city, test_size=0.3)
train_X_256_ci,valid_X_256_ci,train_label_256_ci,valid_label_256_ci = train_test_split(data_256, train__y_labels_city, test_size=0.3)
train_X_128_ci,valid_X_128_ci,train_label_128_ci,valid_label_128_ci = train_test_split(data_128, train__y_labels_city, test_size=0.3)


#Class
train_X_512_cl,valid_X_512_cl,train_label_512_cl,valid_label_512_cl = train_test_split(data_512, train__y_labels_class, test_size=0.3)
train_X_256_cl,valid_X_256_cl,train_label_256_cl,valid_label_256_cl = train_test_split(data_256, train__y_labels_class, test_size=0.3)
train_X_128_cl,valid_X_128_cl,train_label_128_cl,valid_label_128_cl = train_test_split(data_128, train__y_labels_class, test_size=0.3)



In [ ]:
#approach 2 for splitting
x_train_comp = np.stack([data_512,data_256,data_128], axis=1)


In [ ]:
x_train_city, x_test_city, y_train_city, y_test_city = train_test_split(x_train_comp, train__y_labels_city, test_size = 0.3)
x_train_class, x_test_claa, y_train_class, y_test_class = train_test_split(x_train_comp, train__y_labels_class, test_size = 0.3)



#### Test of models individually

In [ ]:
#test of model using data_512 and city clasiification

model_1 = Sequential()
model_1.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu', input_shape=(128, 128, 3)))
model_1.add(Conv2D(32, (3,3), activation='relu'))
model_1.add(MaxPooling2D((2,2)))  
model_1.add(Dropout(0.1)) 
model_1.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model_1.add(Conv2D(64, (3,3), activation='relu'))
model_1.add(MaxPooling2D((2,2)))
model_1.add(Dropout(0.1))
model_1.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model_1.add(Conv2D(128, (3,3), activation='relu'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling2D((2,2)))
model_1.add(Dropout(0.1))
model_1.add(Conv2D(512, (5,5), padding='same', activation='relu'))
model_1.add(Conv2D(512, (5,5), activation='relu'))
model_1.add(MaxPooling2D((4,4)))
model_1.add(Dropout(0.1))

model_1.add(Flatten())
model_1.add(Dense(1024, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(5, activation='softmax'))

In [ ]:
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_1.fit(data_512, train__y_labels_city, batch_size=1, epochs=1)

28195/28195 [==============================] - 2939s 104ms/step - loss: 1.6709 - accuracy: 0.2773


### Creating three models with three different inputs

In [ ]:
model_1 = Sequential()
model_1.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu', input_shape=(128, 128, 3)))
model_1.add(Conv2D(32, (3,3), activation='relu'))
model_1.add(MaxPooling2D((2,2)))  
model_1.add(Dropout(0.1)) 
model_1.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model_1.add(Conv2D(64, (3,3), activation='relu'))
model_1.add(MaxPooling2D((2,2)))
model_1.add(Dropout(0.1))
model_1.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model_1.add(Conv2D(128, (3,3), activation='relu'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling2D((2,2)))
model_1.add(Dropout(0.1))
model_1.add(Conv2D(512, (5,5), padding='same', activation='relu'))
model_1.add(Conv2D(512, (5,5), activation='relu'))
model_1.add(MaxPooling2D((4,4)))
model_1.add(Dropout(0.1))

model_1.add(Flatten())
model_1.add(Dense(1024, activation='relu'))




model_2 = Sequential()
model_2.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu', input_shape=(128, 128, 3)))
model_2.add(Conv2D(32, (3,3), activation='relu'))
model_2.add(MaxPooling2D((2,2)))  
model_2.add(Dropout(0.1)) 
model_2.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model_2.add(Conv2D(64, (3,3), activation='relu'))
model_2.add(MaxPooling2D((2,2)))
model_2.add(Dropout(0.1))
model_2.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model_2.add(Conv2D(128, (3,3), activation='relu'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D((2,2)))
model_2.add(Dropout(0.1))
model_2.add(Conv2D(512, (5,5), padding='same', activation='relu'))
model_2.add(Conv2D(512, (5,5), activation='relu'))
model_2.add(MaxPooling2D((4,4)))
model_2.add(Dropout(0.1))

model_2.add(Flatten())
model_2.add(Dense(1024, activation='relu'))


model_3 = Sequential()
model_3.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu', input_shape=(128, 128, 3)))
model_3.add(Conv2D(32, (3,3), activation='relu'))
model_3.add(MaxPooling2D((2,2)))  
model_3.add(Dropout(0.1)) 
model_3.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model_3.add(Conv2D(64, (3,3), activation='relu'))
model_3.add(MaxPooling2D((2,2)))
model_3.add(Dropout(0.1))
model_3.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model_3.add(Conv2D(128, (3,3), activation='relu'))
model_3.add(Activation('relu'))
model_3.add(MaxPooling2D((2,2)))
model_3.add(Dropout(0.1))
model_3.add(Conv2D(512, (5,5), padding='same', activation='relu'))
model_3.add(Conv2D(512, (5,5), activation='relu'))
model_3.add(MaxPooling2D((4,4)))
model_3.add(Dropout(0.1))

model_3.add(Flatten())
model_3.add(Dense(1024, activation='relu'))

### Concatenate three models into one dense vector

In [ ]:
Concatenated = Concatenate([model_1.outputs[0], model_2.outputs[0],model_3.outputs[0]])

In [ ]:
#City
model_classifier1 = Sequential()
model_classifier1.add(Concatenated)
model_classifier1.add(Flatten())
model_classifier1.add(Dense(2048, activation='relu'))
model_classifier1.add(Dropout(0.1))
model_classifier1.add(Dense(5, activation='softmax'))

#Class
model_classifier2 = Sequential()
model_classifier2.add(Concatenated)
model_classifier2.add(Flatten())
model_classifier2.add(Dense(2048, activation='relu'))
model_classifier2.add(Dropout(0.1))
model_classifier2.add(Dense(18, activation='softmax'))

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0004)

#City
model_classifier1.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['f1_m'])
#Class
model_classifier2.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['f1_m'])

In [ ]:
model_classifier1.fit(x_train_comp, train__y_labels_city, batch_size=1, epochs=1)



ValueError: in user code:

    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/anaconda3/lib/python3.8/site-packages/keras/layers/merge.py", line 498, in build
        raise ValueError(

    ValueError: Exception encountered when calling lay